# 9. Keras and deep learning
In this lab, we will learn how to use Keras to build deep learning models. We will use Keras to build a LSTM model for sentiment classification and a CNN model for digit recognition.
You need to put in the code to complete the models in the blocks marked with `## YOUR CODE HERE` and `## END OF YOUR CODE`.

## Installation
Before you can start using Keras, you'll need to install TensorFolw, which includes Keras as part of its core library.
```bash
source activate {your_env}
pip install tensorflow
pip install keras
```

## Basics of Keras
Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

### 1. Initialize a model
Start by creating a Sequential model and adding layers to it.
```python
from keras.models import Sequential
from keras.layers import Dense

# Initialize a model
model = Sequential()

# Add layers to the model
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

# this is equivalent to the above
#model = Sequential([
#    Dense(64, activation='relu', input_dim=100),
#    Dense(10, activation='softmax')
#])
```


### 2. Compile the model
Compile the model with the appropriate loss function and optimizer.
```python
model.compile(loss='categorical_crossentropy', # loss function, binary_crossentropy for binary classification
              optimizer='sgd', # stochastic gradient descent
              metrics=['accuracy'])
```


### 3. Train the model
Train the model with the training data.
```python
x_train = np.random.random((1000, 100))
y_train = np.random.randint(2, size=(1000, 10))
model.fit(x_train, y_train, epochs=5, batch_size=32)
```


### 4. Evaluate the model
Evaluate the model with the test data.
```python
x_test = np.random.random((100, 100))
y_test = np.random.randint(2, size=(100, 10))
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)
```


## Keras LSTM for IMDB sentiment classification
The IMDB dataset is in `datasets/` of this repository. Use the following code the load the dataset and write a LSTM model to classify the sentiment of the reviews.
```python
import pandas as pd    # to load dataset
import nltk
from nltk.corpus import stopwords   # to get a collection of stopwords

data = pd.read_csv('../datasets/IMDB.csv')

custom_path = '../datasets/'

# Append your custom path to the NLTK data path
nltk.data.path.append(custom_path)

nltk.download('stopwords', download_dir=custom_path)
english_stops = set(stopwords.words('english'))

x_data = data['review']       # Reviews/Input
y_data = data['sentiment']    # Sentiment/Output
# PRE-PROCESS REVIEW
x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
```


The tokenization of the reviews is done by the following code:
```python
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=10000)    # num_words is the number of words to keep based on word frequency
tokenizer.fit_on_texts(x_data)            # fit tokenizer to our training text data

# retrieve the word index
word_index = tokenizer.word_index

x_data = tokenizer.texts_to_sequences(x_data)  # convert our text data to sequence of numbers
```


Now, complete the following code to create a LSTM model for the IMDB sentiment classification.

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, LSTM, Dense, GRU
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils

# Pad sequences to ensure uniform input size
max_length = 100  # You can choose a different length
x_data = pad_sequences(x_data, maxlen=max_length)

# Convert sentiments to numerical labels
y_data = np.where(y_data == 'positive', 1, 0)

# Split data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

## YOUR CODE HERE
# Build the RNN model

# Compile the model

# Train the model

## END OF YOUR CODE

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc}")

## Keras CNN for Digit Recognition
In lab 5, we use the digit dataset. Now, we will use the same dataset to train a CNN model to recognize the digits.
```python
import pandas as pd

X_train = pd.read_csv('../datasets/digits/Digits_X_train.csv').values
y_train = pd.read_csv('../datasets/digits/Digits_y_train.csv').values
X_test  = pd.read_csv('../datasets/digits/Digits_X_test.csv').values
y_test  = pd.read_csv('../datasets/digits/Digits_y_test.csv').values
```

Complete the following code to create a CNN model for the digit recognition.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, Flatten, MaxPooling2D

# Reshape the data to 8 * 8 * 1
X_train = X_train.reshape(X_train.shape[0], 8, 8, 1)
X_test = X_test.reshape(X_test.shape[0], 8, 8, 1)

## YOUR CODE HERE
# Create the model

# Print the model summary
print(model.summary())

# Compile the model

# Train the model
## END OF YOUR CODE

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Accuracy: ", accuracy)